In [8]:
import requests
from bs4 import BeautifulSoup

# URL to scrape
url = "https://www.holaislascanarias.com/experiencias/las-islas-canarias-te-esperan/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.content, 'html.parser')

# Retrieve all text from the page
page_text = soup.get_text()

filtered_text = ' '.join([sentence for sentence in page_text.split('\n') if len(sentence.split()) >= 8])

# Split the filtered text into a list of sentences
text_list = filtered_text.split('. ')

print(text_list)

['Las Islas Canarias te esperan  | Hola Islas Canarias Ocho islas en el océano Atlántico donde desconectar, alejarte de la rutina y regresar con nuevas energías', 'Descubre su naturaleza volcánica, relájate en playas paradisíacas, adéntrate en bosques milenarios, vuela con tu mirada sobre barrancos y acantilados de vértigo, refléjate y sumérgete en aguas cristalinas… Disfruta de una variedad de paisajes extraordinaria y vive infinitas experiencias', '¡Ah!, acuérdate de la gastronomía, la tradicional y la nueva liderada por prestigiosos chefs; y tampoco olvides conocer el valioso patrimonio histórico ni la cultura de las Islas, la de los antiguos pobladores y la actual', 'Y lo mejor: ven cuando quieras, en la época del año que te apetezca', 'Estás en las islas de la eterna primavera', 'Estás en el mejor clima del mundo', 'Ven a conocerlas', 'Horas de playa sobre la arena, tomando el sol y refrescantes chapuzones en el mar, relajan cuerpo y mente', 'Pero también la conexión con los bosqu

In [9]:
import json
from langchain_ollama import OllamaLLM as Ollama

In [23]:
# Función para generar preguntas y respuestas con Llama 3.2
def generate_qa_with_llama(description):
    prompt = f"""
    Genera preguntas relevantes e interesantes basadas en las siguientes oraciones. 
    Asegúrate de que las preguntas sean diversas, significativas y abarquen diferentes aspectos del texto.

    Texto:
    {description}

    Salida esperada: No incluyas nada más al inicio ni al final, únicamente el siguiente formato exacto, respetando minuciosamente las comas, puntos y llaves:
    {{
        "pregunta 1": "respuesta 1",
        "pregunta 2": "respuesta 2",
        "pregunta 3": "respuesta 3"
    }}
    
    EJEMPLO:
    {{
        "¿Donde estan las Islas Canarias?": "En el Oceano Atlantico proximas a la costa noroeste de Africa",
        "¿Que clima existe en las Islas Canarias?": "El clima es subtropical y templado",
        "¿Cuantas islas hay?": "Hay 8 islas principales" 
    }}
    
"""

    try:
        model = Ollama(model="llama3.2")
        response_text = model.invoke(prompt)
        return response_text    
    except Exception as e:
        print(f"Error conectando con Llama 3.2: {e}")
        return None

In [25]:

qa_pairs = []
for text in text_list:
    try:
        response_text = generate_qa_with_llama(text)
        if response_text:
            print(response_text)
            qa_pairs.append(response_text)
            
    except Exception as e:
        print(f"Error generando preguntas y respuestas con Llama 3.2: {e}")


{
    "¿Dónde están las Islas Canarias?": "En el Océano Atlántico, proximas a la costa noroeste de África",
    "¿Cuál es el propósito principal de visitar las Islas Canarias?": "Donde desconectar y regresar con nuevas energías",
    "¿Cuántas islas hay en total?": "8 islas principales",
    "¿Qué tipo de clima caracteriza a las Islas Canarias?": "Subtropical y templado",
    "¿En qué año se creó el concepto de 'Islas Canarias' como destino turístico?": "No especificado en el texto"
}
{
    "¿De qué tipo de paisajes están hablando?:": "Se refiere a una variedad de paisajes naturales como playas, bosques y acantilados",
    "¿Qué experiencias se pueden tener en las Islas Canarias?:", "Puedes disfrutar de diversas actividades al aire libre como buceo, senderismo, avistamiento de aves, etc.",
    "¿Cuál es el clima más característico del lugar?:", "El clima es subtropical y templado con influencia del Atlántico",
    "¿Qué características geológicas tiene la isla?:", "La Isla Canaria tien

In [33]:
# Lista para almacenar los datos formateados
formatted_data = []

# Procesar cada cadena en la lista
for item in qa_pairs:
    try:
        # Intentar cargar cada cadena como JSON
        parsed_data = json.loads(item)
        
        # Extraer preguntas y respuestas
        for question, answer in parsed_data.items():
            entry = {
                "prompt": question.strip(),
                "completion": answer.strip()
            }
            formatted_data.append(entry)
    except json.JSONDecodeError as e:
        print(f"Error procesando la línea: {item}")
        print(f"Detalles del error: {e}")

# Guardar el resultado en un archivo JSONL
output_file = "formatted_data.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    for entry in formatted_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Archivo JSONL generado: {output_file}")

Error procesando la línea: {
    "¿De qué tipo de paisajes están hablando?:": "Se refiere a una variedad de paisajes naturales como playas, bosques y acantilados",
    "¿Qué experiencias se pueden tener en las Islas Canarias?:", "Puedes disfrutar de diversas actividades al aire libre como buceo, senderismo, avistamiento de aves, etc.",
    "¿Cuál es el clima más característico del lugar?:", "El clima es subtropical y templado con influencia del Atlántico",
    "¿Qué características geológicas tiene la isla?:", "La Isla Canaria tiene una naturaleza volcánica, con montañas y valles erosionados por el agua y la lluvia",
    "¿Qué tipo de flora y fauna se puede encontrar en las Islas Canarias?:", "Se pueden encontrar especies endémicas como el cardón, el pino canario y aves como el águila y el halcón",
    "¿Cuál es la bestia marina más grande que se puede encontrar?:", "La ballena azul es una de las especies más grandes que se pueden encontrar en las aguas de las Islas Canarias"
}
Detalle